In [2]:
!conda install -c conda-forge geopy --yes 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /Users/yogirtech/software/anaconda3:

The following NEW packages will be INSTALLED:

    conda-package-handling: 1.6.0-py36h67ae4cd_2  conda-forge
    gdbm:                   1.18-h55c7030_1       conda-forge
    geographiclib:          1.50-py_0             conda-forge
    geopy:                  1.22.0-pyh9f0ad1d_0   conda-forge
    krb5:                   1.17.1-h1752a42_0     conda-forge
    libcurl:                7.69.1-hc0b9707_0     conda-forge
    pypy3.6:                7.3.1-h01e4c1a_1      conda-forge
    python_abi:             3.6-1_pypy36_pp73     conda-forge
    tqdm:                   4.46.0-pyh9f0ad1d_0   conda-forge

The following packages will be UPDATED:

    bzip2:                  1.0.6-h92991f9_1                  --> 1.0.8-h0b31af3_2             conda-forge
    conda:                  4.3.30-py36h173c244_0             --> 4.8.3-py36hc560c46_2  

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#### Get the Postal Code data and create an empty data frame

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)
df.head()

,Postalcode,Borough,Neighborhood


#### Process the data and add it to the data frame

In [26]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Remove the rows with values "Not assigned"

In [27]:
df = df[df.Borough != "Not assigned"]
df.reset_index(drop=True, inplace=True) # drop the original index
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Append the Neighborhood data for a all the Postal Codes and then drop the duplicate entries for repeated postalcodes

In [24]:
# Combine the Neighborhood values for the Postalcodes
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhoods'},inplace=True)

# Merge the temp_df with original data frame on Postalcode 
toronto_df = pd.merge(df, temp_df, on='Postalcode')

# drop the column Neighborhood as we have the new values in Neighborhoods column
toronto_df.drop(['Neighborhood'],axis=1,inplace=True)

# drop the duplicate entries
toronto_df.drop_duplicates(inplace=True)

# Rename the Neighborhoods column that has the combined values ro Neighborhood
toronto_df.rename(columns={'Neighborhoods':'Neighborhood'},inplace=True)

# Reset the index
toronto_df.reset_index(drop=True, inplace=True) # drop the original index
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
2,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
3,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [25]:
toronto_df.shape

(103, 3)